# Rapprochement DPE juillet 2025

In [1]:
import pandas as pd

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
usecols = ['numero_dpe', 'identifiant_ban', 'code_postal_ban', 'code_postal_brut', 'id_rnb', 'provenance_id_rnb']
dtype={'numero_dpe': 'string', 'identifiant_ban': 'string', 'code_postal_ban': 'string', 'code_postal_brut': 'string', 'id_rnb': 'string', 'provenance_id_rnb': 'string'}

# usecols = ['numero_dpe', 'id_rnb', 'provenance_id_rnb']
# dtype={'numero_dpe': 'string', 'id_rnb': 'string', 'provenance_id_rnb': 'string'}

In [7]:
df_tertiaire = pd.read_csv('notebooks/rapprochements/DPE/2025/data/dpe01tertiaire.csv', sep=',', usecols=usecols, dtype=dtype)
df_tertiaire['file'] = 'tertiaire'

In [8]:
df_existant = pd.read_csv('notebooks/rapprochements/DPE/2025/data/dpe03existant.csv', sep=',', usecols=usecols, dtype=dtype)
df_existant['file'] = 'existant'

KeyboardInterrupt: 

In [ ]:
df_neuf = pd.read_csv('notebooks/rapprochements/DPE/2025/data/dpe02neuf.csv', sep=',', usecols=usecols, dtype=dtype)
df_neuf['file'] = 'neuf'

In [ ]:
df = pd.concat([df_tertiaire, df_neuf, df_existant])

In [ ]:
df.shape

(13742029, 7)

In [ ]:
df.head()

,numero_dpe,id_rnb,provenance_id_rnb,code_postal_ban,identifiant_ban,code_postal_brut,file
0,2269T2615953W,<NA>,<NA>,69440,69228_0086_00707,69440,tertiaire
1,2313T1646022S,<NA>,<NA>,13360,13086_0035_00008,13360,tertiaire
2,2404T3934296Y,<NA>,<NA>,04220,04197_0231,04220,tertiaire
3,2394T2686905U,<NA>,<NA>,94420,94059_0530_00165,94420,tertiaire
4,2360T0127880L,<NA>,<NA>,60650,60073_0075_00007,60650,tertiaire


In [ ]:
# create 100 subfiles in the dpe_existant folder, using the number of rows in the dataframe
for i in range(0, 100):
    df.iloc[i*df.shape[0]//100:(i+1)*df.shape[0]//100].to_csv(f'notebooks/rapprochements/DPE/2025/sub_files/dpe-{i}.csv', index=False)

: 

In [ ]:
# le rapprochement

import os
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
from django.db import connection
from concurrent.futures import ThreadPoolExecutor
import numpy as np

def get_rnb_id(row):
    cursor = connection.cursor()

    if row['code_postal_ban'] != row['code_postal_brut']:
        return []
    
    ban_id = row['identifiant_ban']

    sql = f"""
            with rnb_ids as (
            select
                rnb_id
            from
                batid_buildingaddressesreadonly bb
            left join batid_building bb2 on
                bb2.id = bb.building_id
            where
                address_id = '{ban_id}'
                and ST_AREA(shape::geography) > 25)
            select
                array_agg(rnb_id)
            from
                rnb_ids;
    """
    cursor.execute(sql)
    result = cursor.fetchone()
    return result[0] if result[0] is not None else []

def execute(df):
    df_copy = df.copy()
    df_copy['rnb_id_rappro'] = df_copy.apply(get_rnb_id, axis=1)
    return df_copy


def process_sub_file(i):
    print(f"processing file {i}")
    if not os.path.exists(f'notebooks/rapprochements/DPE/2025/sub_files_results/dpe-{i}-result.csv'):
        df_sub_file = pd.read_csv(f'notebooks/rapprochements/DPE/2025/sub_files/dpe-{i}.csv', sep=',')
        max_workers = 50
        dfs = np.array_split(df_sub_file, max_workers)

        with ThreadPoolExecutor(max_workers=max_workers) as executor:
            results_sub_file = executor.map(execute, dfs)
            df_result = pd.concat(results_sub_file)
            df_result.to_csv(f'notebooks/rapprochements/DPE/2025/sub_files_results/dpe-{i}-result.csv', index=False)

for i in range(100):
    process_sub_file(i)

processing file 0
processing file 1
processing file 2
processing file 3
processing file 4
processing file 5
processing file 6
processing file 7
processing file 8
processing file 9
processing file 10
processing file 11
processing file 12
processing file 13
processing file 14
processing file 15
processing file 16
processing file 17
processing file 18
processing file 19
processing file 20
processing file 21
processing file 22
processing file 23
processing file 24
processing file 25
processing file 26
processing file 27
processing file 28
processing file 29
processing file 30
processing file 31
processing file 32
processing file 33
processing file 34
processing file 35
processing file 36
processing file 37
processing file 38
processing file 39
processing file 40
processing file 41
processing file 42
processing file 43
processing file 44
processing file 45
processing file 46
processing file 47
processing file 48
processing file 49
processing file 50
processing file 51
processing file 52
pro

In [ ]:
# aggregation des sous fichiers de résultats en un seul
dfs_result = []

for i in range(100):
    df_result = pd.read_csv(f'notebooks/rapprochements/DPE/2025/sub_files_results/dpe-{i}-result.csv', usecols=['numero_dpe', 'file', 'rnb_id_rappro', 'id_rnb', 'provenance_id_rnb'])
    dfs_result.append(df_result)

df_final = pd.concat(dfs_result)
df_final['rnb_id_rappro'] = df_final['rnb_id_rappro'].apply(eval)


/tmp/ipykernel_2074/3264209841.py:4: DtypeWarning: Columns (1,2) have mixed types. Specify dtype option on import or set low_memory=False.
  df_result = pd.read_csv(f'notebooks/rapprochements/DPE/2025/sub_files_results/dpe-{i}-result.csv', usecols=['numero_dpe', 'file', 'rnb_id_rappro', 'id_rnb', 'provenance_id_rnb'])
/tmp/ipykernel_2074/3264209841.py:4: DtypeWarning: Columns (1,2) have mixed types. Specify dtype option on import or set low_memory=False.
  df_result = pd.read_csv(f'notebooks/rapprochements/DPE/2025/sub_files_results/dpe-{i}-result.csv', usecols=['numero_dpe', 'file', 'rnb_id_rappro', 'id_rnb', 'provenance_id_rnb'])
/tmp/ipykernel_2074/3264209841.py:4: DtypeWarning: Columns (1,2) have mixed types. Specify dtype option on import or set low_memory=False.
  df_result = pd.read_csv(f'notebooks/rapprochements/DPE/2025/sub_files_results/dpe-{i}-result.csv', usecols=['numero_dpe', 'file', 'rnb_id_rappro', 'id_rnb', 'provenance_id_rnb'])
/tmp/ipykernel_2074/3264209841.py:4: Dty

In [ ]:
# sauvegarde des résultats
df_final.to_csv('notebooks/rapprochements/DPE/2025/results_DPE_RNB.csv', index=False)